In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, month_plot
from arch.unitroot import ADF, KPSS
from pmdarima import auto_arima
import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima.model import ARIMA, ARIMAResults
from statsmodels.tsa.holtwinters import Holt
from scipy.stats.mstats import winsorize
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

import evds as ev

In [3]:
with open("/Users/neseuygun/Desktop/evds_api_key.txt") as file:
    api = file.read()

evds = ev.evdsAPI(api)

dolar = evds.get_data(["TP.DK.USD.S.YTL"], startdate="01-01-2010", enddate="01-01-2024", frequency=5) #5 ile aylık veri alıyoruz
endeks = evds.get_data(["TP.MK.F.BILESIK"], startdate="01-01-2010", enddate="01-01-2024", frequency=5)
tufe = evds.get_data(["TP.FG.J0"], startdate="01-01-2010", enddate="01-01-2024", frequency=5)

In [9]:
print(f"dolar: {len(dolar)} endeks: {len(endeks)} tufe: {len(tufe)}")

dolar: 169 endeks: 169 tufe: 168


In [10]:
dolar = dolar[:-1]
endeks = endeks[:-1]
print(f"dolar: {len(dolar)} endeks: {len(endeks)} tufe: {len(tufe)}")

dolar: 168 endeks: 168 tufe: 168


In [15]:
for i in [dolar, endeks, tufe]:
    print(i.head())


    Tarih  TP_DK_USD_S_YTL
0  2010-1         1.473400
1  2010-2         1.512830
2  2010-3         1.535678
3  2010-4         1.495048
4  2010-5         1.542210
    Tarih  TP_MK_F_BILESIK
0  2010-1           540.28
1  2010-2           493.84
2  2010-3           569.89
3  2010-4           587.63
4  2010-5           540.34
    Tarih  TP_FG_J0
0  2010-1    174.07
1  2010-2    176.59
2  2010-3    177.62
3  2010-4    178.68
4  2010-5    178.04


In [19]:
dolar.rename(columns={dolar.columns[1]:"Dolar/TL"}, inplace=True)
endeks.rename(columns={endeks.columns[1]:"Bist100"}, inplace=True)
tufe.rename(columns={tufe.columns[1]:"Tufe"}, inplace=True)

In [20]:
endeks

,Tarih,Bist100
0,2010-1,540.28
1,2010-2,493.84
2,2010-3,569.89
3,2010-4,587.63
4,2010-5,540.34
...,...,...
163,2023-8,7917.93
164,2023-9,8334.94
165,2023-10,7514.10
166,2023-11,7948.60
